In [1]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import itertools
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import interp1d

In [2]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times"] + plt.rcParams["font.serif"]
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['mathtext.rm'] = 'Serif'
mpl.rcParams['mathtext.it'] = 'Serif'
mpl.rcParams['mathtext.bf'] = 'Serif'
mpl.rcParams['text.usetex']= False
plt.rc('axes', linewidth=2)
plt.rc('font', weight='bold')

In [3]:
!pwd

/home/u1159830/workspace/MESA-tests


Get grid data

In [4]:
def get_data(index, grid_dir):
    prof_index_i = np.loadtxt(f"{grid_dir}/profiles/profiles_{index}.index", skiprows=1, dtype=int)
    freqs_i = []
    n_profs_i = []
    freq_files = sorted(glob.glob(f"{grid_dir}/gyre/freqs_track_{index}/profile*-freqs.dat"), key=lambda x: int(os.path.basename(x).split('.')[0].split('profile')[1].split('-')[0]))
    for file in freq_files:
        freqs_i.append(pd.read_table(file, skiprows=5, sep='\s+'))
        n_profs_i.append(int(file.split('profile')[-1].split('-')[0]))
    hist_i = pd.read_table(f"{grid_dir}/histories/history_{index}.data", skiprows=5, sep='\s+')
    return hist_i, freqs_i, n_profs_i, prof_index_i


def get_all(grid_dir="../MESA-grid/grid_urot"):
    hists = []
    freqs = []
    n_profs = []
    prof_index = []

    for i in range(1, 14874):
        try:
            hist_i, freqs_i, n_profs_i, prof_index_i = get_data(i, grid_dir=grid_dir)
            hists.append(hist_i)
            freqs.append(freqs_i)
            n_profs.append(n_profs_i)
            prof_index.append(prof_index_i)
        except:
            print(f"Failed to get data for grid index {i}")
    return hists, freqs, n_profs, prof_index


In [5]:
data = get_all("../MESA-grid/grid_urot")

$\Delta \nu$ and $\epsilon$

In [5]:
def fit_radial(ts, degree=0):
    """
    Fits a straight line to the radial mode frequencies. Optionally, can be used on non-radial modes.
    Only modes with radial orders 5-9 are used, as the ridges should be vertical here.
    
    Input: Theoretical (or observed) spectrum in pandas df format; mode degree to be used (default 0 = radial)
    Output: The length of the series used, and the slope, intercept, r_value, p_value, and std_err of the line.
    """
    n_min, n_max = 5, 9
    try:
        vert_freqs = ts.query("n_g == 0").query(f"l=={degree}").query(f"n_pg>={n_min}").query(f"n_pg<={n_max}")[["n_pg","Re(freq)"]].values
    except:
        vert_freqs = ts.query(f"l_obs=={degree}").query(f"n_obs>={n_min}").query(f"n_obs<={n_max}")[["n_obs","f_obs"]].values
    if len(vert_freqs>0):
        slope, intercept, r_value, p_value, std_err = linregress(vert_freqs[:,0], vert_freqs[:,1])
    else:
        slope, intercept, r_value, p_value, std_err = np.zeros(5)
    return len(vert_freqs), slope, intercept, r_value, p_value, std_err

def model_epsilon(ts):
    """
    Calls the fit_radial function to determine the epsilon value for a star's pulsations.
    
    Input: Theoretical (or observed) spectrum in pandas df format.
    Output: Epsilon
    """
    length_rad, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=0)
    eps = intercept/slope
    if length_rad < 3:
        length_dip, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=1)
        if length_dip > length_rad:
            eps = intercept/slope - 0.5 # take the ell=1 values and subtract 0.5 to equal epsilon (ell=0)
    return np.round(eps, 3)

def model_Dnu(ts):
    """
    Calls the fit_radial function to determine the Delta nu value for a star's pulsations.
    
    Input: Theoretical (or observed) spectrum in pandas df format.
    Output: Delta nu
    """
    length_rad, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=0)
    if length_rad < 3:
        length_dip, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=1)
        if length_rad > length_dip:
            # redo radial
            length_rad, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=0)
    Dnu = slope
    return np.round(Dnu, 3)

def get_fit(l, freq):
    Dnu = model_Dnu(freq)
    epsilon = model_epsilon(freq)
    return Dnu, epsilon